<a href="https://colab.research.google.com/github/Abdou-ch-d/Breast-Cancer-Detection/blob/main/Dicom_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# mount drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Install necessary packages 


In [ ]:
# %%capture
# !pip install patchify

In [ ]:
# %%capture
# !pip install SimpleITK

In [ ]:
# %%capture
# !pip install pydicom

In [ ]:
# %%capture
# !pip install opencv-python-headless==4.5.2.52

In [ ]:
# %%capture
# !pip install --upgrade pandas

In [ ]:
# %%capture
# !pip install --upgrade matplotlib

In [ ]:
# %%capture
# !pip install -U albumentations

In [ ]:
# %%capture
# !pip install --force-reinstall albumentations==1.0.3

In [ ]:
# %%capture
# !pip install --force-reinstall opencv-python-headless==4.1.2.30

In [ ]:
# %%capture
# !unzip '/content/drive/MyDrive/INBreast_Dataset/INbreast Release 1.0.zip' -d '/content/Datasets'

In [ ]:
# exit()

# Bib

In [ ]:
import time
import cv2
import colorsys
from tqdm.notebook import tqdm, trange
import albumentations as A
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import random
import os, shutil
from glob import glob
import SimpleITK as sitk

from matplotlib.colors import LinearSegmentedColormap

import pydicom as dicom
from pathlib import Path
from sklearn.model_selection import train_test_split
import shutil
import plistlib
from skimage.draw import polygon
from patchify import patchify

# Functions

**This function loads a osirix xml region as a binary numpy array for INBREAST
  dataset**

return: numpy array where each mass has a different number id

In [ ]:
#@title load_inbreast_mask{form-width:"0.5%"}
def load_inbreast_mask(mask_path, imshape=(4084, 3328), class__=0):
    """
    This function loads a osirix xml region as a binary numpy array for INBREASTdataset
    @mask_path : Path to the xml file
    @class__: mask class, 
              class__ != 0 --> mass
              class__ == 0 --> calcification (default)
    @imshape : The shape of the image as an array e.g. [4084, 3328]
    return: numpy array where each mass has a different number id.

    
    """

    def load_point(point_string):
        x, y = tuple([float(num) for num in point_string.strip('()').split(',')])
        return y, x
        
    i =  0
    mask = np.zeros(imshape)
    if(class__):
      c='Mass'
    else:
      c='Calcification'

    with open(mask_path, 'rb') as mask_file:
        plist_dict = plistlib.load(mask_file, fmt=plistlib.FMT_XML)['Images'][0]
        numRois = plist_dict['NumberOfROIs']
        rois = plist_dict['ROIs']
        assert len(rois) == numRois
        for roi in rois:
            numPoints = roi['NumberOfPoints']
            
            if roi['Name'] == c:
                i+=1
                points = roi['Point_px']
                assert numPoints == len(points)
                points = [load_point(point) for point in points]
                if len(points) <= 2:
                    for point in points:
                        mask[int(point[0]), int(point[1])] = i
                else:
                    x, y = zip(*points)
                    x, y = np.array(x), np.array(y)
                    poly_x, poly_y = polygon(x, y, shape=imshape)
                    mask[poly_x, poly_y] = i
        

    return mask

**Convert a mask into albumentations format**


mass_mask : numpy array mask where each pixel correspond to a lesion (one pixel id per lesion)

In [ ]:
#@title mask_to_yolo{form-width:"0.5%"}
def mask_to_yolo(mass_mask,c):
    """
    return: a list of list containing masses bounding boxes in YOLO coordinates:
            <x> = <absolute_x> / <image_width>
            <y> = <absolute_y> / <image_height>
            <height> = <absolute_height> / <image_height>
            <width> = <absolute_width> / <image_width>
    """
   
    ids=np.unique(mass_mask)[1:]
    res = []
    height, width = mass_mask.shape
    nbr_mass = len(np.unique(mass_mask))-1
    
    
    for i in range(nbr_mass):
        mask = mass_mask.copy()
        mask[mass_mask!=ids[i]]=0
        #find contours of each mass
        cnts, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        #create a bbox around the contours
        x, y, w, h = cv2.boundingRect(cnts[0])
        # print("xywh\n",x,y,w,h)
        #convert to yolo coordinate system
        # x = x+w//2 -1
        # y= y+h//2 -1
        # res.append([x/width,y/height,w/width,h/height, 'Calcification'])
        # print("c= ",c)
        if(c==1):
          #c=1 so res[5]=Masse
          res.append([x,y,w,h,1])
          # print("res of Masses\n",res)
        elif(c==0):
          #c=0 so res[5]=Calcification
          res.append([x,y,w,h,0])
          # print("res of Calcifications\n",res)
    return res

**write  the list of bbox into txt file**

In [ ]:
#@title bbox_to_txt{form-width:"0.5%"}
def bbox_to_txt(bboxes):
    """
    bboxes : numpy array of bounding boxes 
    return : a string for each object in new line: <object-class> <x> <y> <width> <height>
    """
    txt=''
    for l in bboxes:
        l = [str(x) for x in l[:4]]
        l = ' '.join(l)
        if(l[5]==0):
          txt += '0 ' + l + '\n'
        # else:
        #   txt += '1 ' + l + '\n'
    return txt

# Image processing

**Crop breast ROI from image**  

This can be done with OpenCV and Otsu’s thresholding technique

In [ ]:
#@title crop{form-width:"0.5%"}
def crop(img, mask_mass, mask_cal):
    """
    @img : numpy array image
    @mask : numpy array mask of the lesions
    return: numpy array of the ROI extracted for the image, 
            numpy array of the ROI extracted for the breast mask,
            numpy array of the ROI extracted for the masses mask
    """
    # Otsu's thresholding after Gaussian filtering
    blur = cv2.GaussianBlur(img,(5,5),0)
    blur=blur.astype(np.uint8)
    _, breast_mask = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    cnts, _ = cv2.findContours(breast_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt = max(cnts, key = cv2.contourArea)
    x, y, w, h = cv2.boundingRect(cnt)
    return img[y:y+h, x:x+w], breast_mask[y:y+h, x:x+w], mask_mass[y:y+h, x:x+w], mask_cal[y:y+h, x:x+w]

**3 preprocessing steps**


*   Truncation normalization
*   Image enhancement
*   Image synthesizing




**Clip and normalize pixels in the breast ROI**

After cropping, the image is still composed of many **black pixels**. This may have negative effects on the detection if we normalize the image as it is (because the breast region will appear less intense). Here the goal is to normalize the **intensity distribution of the pixels** in the breast region.

In [ ]:
#@title truncation_normalization{form-width:"0.5%"}
def truncation_normalization(img, mask):
    """
    *img : numpy array image
    *mask : numpy array mask of the breast
    return: numpy array of the normalized image
    """
    Pmin = np.percentile(img[mask!=0], 5)
    Pmax = np.percentile(img[mask!=0], 99)
    truncated = np.clip(img,Pmin, Pmax)  
    normalized = (truncated - Pmin)/(Pmax - Pmin)
    normalized[mask==0]=0
    return normalized

**Image enhancement**

To enhance the contrast of the breast region and so the mass lesion, Contrast Limited Adaptive Histogram Equalization (**CLAHE**) algorithm is applied with clip limit



In [ ]:
#@title CLAHE{form-width:"1%"}
def clahe(img, clip=2.0, tile=(8, 8),):
    
    '''
    This function applies the Contrast-Limited Adaptive
    Histogram Equalisation filter to a given image.
    
    Parameters
    ----------
    img : {numpy.ndarray}
        The image to edit.
    clip : {int or floa}
        Threshold for contrast limiting.
    tile : {tuple (int, int)}
        Size of grid for histogram equalization. Input
        image will be divided into equally sized
        rectangular tiles. `tile` defines the number of
        tiles in row and column.
    
    Returns
    -------
    clahe_img : {numpy.ndarray}
        The edited image.
    '''
    
    # Convert to uint8.
    # img = skimage.img_as_ubyte(img)

    clahe_create = cv2.createCLAHE(clipLimit=clip, 
                                   tileGridSize=tile,
                                   )
    clahe_img = clahe_create.apply(img)

    return clahe_img

**Image synthesizing**

Create a 3-channel image composed of the truncated and normalized image,
the contrast enhanced image with clip limit 1, 
and the contrast enhanced image with clip limit 2 

In [ ]:
#@title synthetized_images{form-width:"0.5%"}
def synthetized_images(patient_id, DatasetPath):
    """
    @patient_id : patient id to recover image and mask in the dataset
    return: numpy array of the breast region
            numpy array of the synthetized images
            numpy array of the masses mask
    """
    image_path = glob(DatasetPath+'AllDICOMs/'+str(patient_id)+'*.dcm')[0]

    calcification_mask = load_inbreast_mask(DatasetPath+'AllXML/'+str(patient_id)+'.xml', class__=0)
    mass_mask = load_inbreast_mask(DatasetPath+'AllXML/'+str(patient_id)+'.xml', class__=1)

    ds = dicom.dcmread(image_path)
    pixel_array_numpy = ds.pixel_array
    
    breast, mask, mass_mask ,calcification_mask= crop(pixel_array_numpy, mass_mask,calcification_mask)

    # normalized = truncation_normalization(breast, mask)
   
    normalized=cv2.normalize(
        breast,
        None,
        alpha=0,
        beta=255,
        norm_type=cv2.NORM_MINMAX,
        dtype=cv2.CV_8U,
    )
    
    cl1 = clahe(normalized, 1.0)
    cl2 = clahe(normalized, 2.0)
    # synthetized = cv2.merge((np.array(normalized*255, dtype=np.uint8),cl1,cl2))

    synthetized = np.moveaxis(np.array([normalized, cl1, cl2]), 0, -1)
    mass_mask= mass_mask.astype(np.uint8)
    calcification_mask= calcification_mask.astype(np.uint8)
    
    return breast, synthetized, mass_mask, calcification_mask

p:probability of applying the transform

In [ ]:
#@title Transform{form-width:"0.5%"}
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=90,interpolation=1,border_mode=cv2.BORDER_CONSTANT,value=0, mask_value=0,p=0.5)
    # A.SafeRotate(border_mode=cv2.BORDER_CONSTANT, value=0, mask_value=0,p=0.5)  #.geometric.rotate.SafeRotate
    # A.ShiftScaleRotate(scale_limit = 0, rotate_limit=180, p=1, border_mode=0  )
],
# bbox_params=A.BboxParams(format='yolo' , min_visibility=0.1),

)

In [ ]:
#@title Main PreProcess Function{form-width:"0.5%"}
def PreprocessDicom(set_ids, size, nb_aug, DatasetPath, GenertedDatasetPath):
  cntr = 0
  dim=(size,size)
  arry_bboxes=[]
  for patient_id in tqdm(set_ids):
          original, synthetized, mass_mask ,cal_mask= synthetized_images(patient_id, DatasetPath)

          #Resizing an image needs a way to calculate pixel values for the new image from the original one.
          synthetized = cv2.resize(synthetized, dim, interpolation = cv2.INTER_AREA)  #synthetized
          mass_mask = cv2.resize(mass_mask, dim, interpolation = cv2.INTER_NEAREST).astype(np.uint8)
          cal_mask = cv2.resize(cal_mask, dim, interpolation = cv2.INTER_NEAREST).astype(np.uint8)

          
          # bboxes = mask_to_yolo(mass_mask)

          # if(cntr):
          #       arry_imgs=np.append(arry_imgs,[synthetized],axis=0)
          #       arry_masks=np.append(arry_masks,[mass_mask],axis=0)
          # else:
          #       arry_imgs=np.array([synthetized])
          #       arry_masks=np.array([mass_mask])
          # arry_bboxes.append(bboxes)
          
          #Saving instead of append
          
          sitk.WriteImage(sitk.GetImageFromArray(synthetized),
                          f"{GenertedDatasetPath}Images/{cntr}.dcm"
                          )
    
          sitk.WriteImage(sitk.GetImageFromArray(mass_mask),
                          f"{GenertedDatasetPath}Masks_masses/{cntr}.dcm"
                          )
          
          sitk.WriteImage(sitk.GetImageFromArray(cal_mask),
                          f"{GenertedDatasetPath}Masks_cal/{cntr}.dcm"
                          )
          cntr+=1


          for i in range(nb_aug):

              r=random.randrange(1000)
              random.seed(r)
            
              try:
                  transformed_cal= transform(image=synthetized, mask=cal_mask)
                  random.seed(r)
                  transformed_mass= transform(image=synthetized, mask=mass_mask)
              except:
                    print('patient_id_______',patient_id)
                          
              transformed_image = transformed_cal['image']
              transformed_Masks_cal = transformed_cal['mask']
              transformed_Masks_mass=transformed_mass['mask']

              # bboxes = mask_to_yolo(transformed_Masks)
              
              sitk.WriteImage(sitk.GetImageFromArray(transformed_image),
                              f"{GenertedDatasetPath}Images/{cntr}.dcm"
                              )
    
              sitk.WriteImage(sitk.GetImageFromArray(transformed_Masks_mass),
                              f"{GenertedDatasetPath}Masks_masses/{cntr}.dcm"
                              )
              sitk.WriteImage(sitk.GetImageFromArray(transformed_Masks_cal),
                              f"{GenertedDatasetPath}Masks_cal/{cntr}.dcm"
                              )
              # arry_imgs=np.append(arry_imgs,[transformed_image],axis=0)
              # arry_masks=np.append(arry_masks,[transformed_Masks],axis=0)
              # arry_bboxes.append(bboxes)
              
              cntr+=1

  # arry_masks= np.where(arry_masks>0, 1, 0).astype("float32")
  # arry_imgs= arry_imgs/255
  # return arry_imgs, arry_masks#, arry_bboxes

In [ ]:
#@title Main PreProcess Function V2{form-width:"0.5%"}
def PreprocessDicomV2(set_ids, size, nb_aug, DatasetPath, GenertedDatasetPath):
  cntr = 0
  dim=(size,size)
  arry_bboxes=[]
  for patient_id in tqdm(set_ids):
          original, synthetized, mass_mask ,cal_mask= synthetized_images(patient_id, DatasetPath)

          #Resizing an image needs a way to calculate pixel values for the new image from the original one.
          synthetized = cv2.resize(synthetized, dim, interpolation = cv2.INTER_AREA)  #synthetized
          mass_mask = cv2.resize(mass_mask, dim, interpolation = cv2.INTER_NEAREST).astype(np.uint8)
          cal_mask = cv2.resize(cal_mask, dim, interpolation = cv2.INTER_NEAREST).astype(np.uint8)

          #Merge masks in one
          mass_mask= np.where(mass_mask>0, 2, 0)
          cal_mask= np.where(cal_mask>0, 1, 0)
          mass_mask[cal_mask!=0]=0
          mask=(mass_mask+cal_mask).astype(np.uint8)

          #Saving
          sitk.WriteImage(sitk.GetImageFromArray(synthetized),
                          f"{GenertedDatasetPath}Images/{cntr}.dcm"
                          )

          sitk.WriteImage(sitk.GetImageFromArray(mask),
                          f"{GenertedDatasetPath}Masks/{cntr}.dcm"
                          )
          
          cntr+=1

          #Augmentation
          for i in range(nb_aug):
            
              try:
                  transformed_cal= transform(image=synthetized, mask=mask)
              except:
                    print('patient_id_______',patient_id)
                          
              transformed_image = transformed_cal['image']
              transformed_mask= transformed_cal['mask']
              
              sitk.WriteImage(sitk.GetImageFromArray(transformed_image),
                              f"{GenertedDatasetPath}Images/{cntr}.dcm"
                              )

              sitk.WriteImage(sitk.GetImageFromArray(transformed_mask),
                          f"{GenertedDatasetPath}Masks/{cntr}.dcm"
                          )
              
              cntr+=1

In [ ]:
#@title Main PreProcess Patches Function{form-width:"0.5%"}
def PreprocessDicomPatches(set_ids, size, nb_aug, DatasetPath, GenertedDatasetPath):
  '''
  size: since no resize needed, size now represent patches fixed size (model input size).
  step: by default patchification step equals size to avoid overlap between patches.
  '''
  cntr = 0
  dim=(size,size)
  step=size

  for patient_id in tqdm(set_ids):
          original, synthetized, mass_mask ,cal_mask= synthetized_images(patient_id, DatasetPath)

          img_patches = patchify(synthetized, dim+(3,), step=step)
          mass_mask_patches = patchify(mass_mask, dim, step=step)
          cal_mask_patches = patchify(cal_mask, dim, step=step)

          
          #Saving
          cpt=0
          for i in range(img_patches.shape[0]):
            for j in range(img_patches.shape[1]):
              if img_patches[i,j,0,:,:,0].sum()==0:
                continue

              sitk.WriteImage(sitk.GetImageFromArray(img_patches[i,j,0,...]),
                          f"{GenertedDatasetPath}Images/{cntr}_{cpt}.dcm"
                          )
              sitk.WriteImage(sitk.GetImageFromArray(mass_mask_patches[i,j,...]),
                          f"{GenertedDatasetPath}Masks_masses/{cntr}_{cpt}.dcm"
                          )
              sitk.WriteImage(sitk.GetImageFromArray(cal_mask_patches[i,j,...]),
                          f"{GenertedDatasetPath}Masks_cal/{cntr}_{cpt}.dcm"
                          )
              cpt+=1

          cntr+=1


          #Augmentation
          for i in range(nb_aug):
              cpt=1
              for i in range(img_patches.shape[0]):
                for j in range(img_patches.shape[1]):
                  if img_patches[i,j,0,:,:,0].sum()==0:
                    continue
                    
                  r=random.randrange(1000)
                  random.seed(r)
                
                  try:
                      transformed_cal= transform(image=img_patches[i,j,0,...], mask=cal_mask_patches[i,j,...])
                      random.seed(r)
                      transformed_mass= transform(image=img_patches[i,j,0,...], mask=mass_mask_patches[i,j,...])
                  except:
                        print('patient_id_______',patient_id)
                              
                  transformed_image = transformed_cal['image']
                  transformed_Masks_cal = transformed_cal['mask']
                  transformed_Masks_mass= transformed_mass['mask']

                  
                  sitk.WriteImage(sitk.GetImageFromArray(transformed_image),
                                  f"{GenertedDatasetPath}Images/{cntr}_{cpt}.dcm"
                                  )
        
                  sitk.WriteImage(sitk.GetImageFromArray(transformed_Masks_mass),
                                  f"{GenertedDatasetPath}Masks_masses/{cntr}_{cpt}.dcm"
                                  )
                  sitk.WriteImage(sitk.GetImageFromArray(transformed_Masks_cal),
                                  f"{GenertedDatasetPath}Masks_cal/{cntr}_{cpt}.dcm"
                                  )
                  cpt+=1
                
              cntr+=1

In [ ]:
#@title Main PreProcess Patches Function  V2{form-width:"0.5%"}
def PreprocessDicomPatchesV2(set_ids, size, nb_aug, DatasetPath, GenertedDatasetPath):
  cntr = 0
  dim=(size,size)
  step=size
  arry_bboxes=[]
  for patient_id in tqdm(set_ids):
          original, synthetized, mass_mask ,cal_mask= synthetized_images(patient_id, DatasetPath)


          #Merge masks in one
          mass_mask= np.where(mass_mask>0, 2, 0)
          cal_mask= np.where(cal_mask>0, 1, 0)
          mass_mask[cal_mask!=0]=0
          mask=(mass_mask+cal_mask).astype(np.uint8)

          img_patches = patchify(synthetized, dim+(3,), step=step)
          mask_patches = patchify(mask, dim, step=step)


          #Saving
          cpt=0
          for i in range(img_patches.shape[0]):
            for j in range(img_patches.shape[1]):
              if img_patches[i,j,0,:,:,0].sum()==0:
                continue
                
              sitk.WriteImage(sitk.GetImageFromArray(img_patches[i,j,0,...]),
                          f"{GenertedDatasetPath}Images/{cntr}_{cpt}.dcm"
                          )
              sitk.WriteImage(sitk.GetImageFromArray(mask_patches[i,j,...]),
                          f"{GenertedDatasetPath}Masks/{cntr}_{cpt}.dcm"
                          )
              cpt+=1

          cntr+=1


          #Augmentation
          for i in range(nb_aug):
            cpt=1
            for i in range(img_patches.shape[0]):
              for j in range(img_patches.shape[1]):
                if img_patches[i,j,0,:,:,0].sum()==0:
                  continue
                try:
                    transformed_cal= transform(image=img_patches[i,j,0,...], mask=mask_patches[i,j,...])
                except:
                      print('patient_id_______',patient_id)
                            
                transformed_image = transformed_cal['image']
                transformed_mask= transformed_cal['mask']
                
                sitk.WriteImage(sitk.GetImageFromArray(transformed_image),
                                f"{GenertedDatasetPath}Images/{cntr}_{cpt}.dcm"
                                )

                sitk.WriteImage(sitk.GetImageFromArray(transformed_mask),
                            f"{GenertedDatasetPath}Masks/{cntr}_{cpt}.dcm"
                            )
                cpt+=1
            cntr+=1

# Test Preprocess

**Train set augmentation**

In [ ]:
# from sklearn.model_selection import train_test_split

In [ ]:
# DatasetPath='/content/Datasets/INbreast Release 1.0/'

In [ ]:
# XMLS=DatasetPath+'AllXML/*.xml'
# xml=glob(XMLS)
# Callcification_PATIENT_ID=[x.split('/')[-1].split('.')[0]for x in xml]

# # Split the data into train val & test set
# train_set,test_set  = train_test_split(Callcification_PATIENT_ID, test_size = 0.1, random_state=82)
# # train_set, test_set = train_test_split(train_set, test_size = 0.11, random_state=seed)

# print("train set: {0}\ntest set: {1}".format(len(train_set),
#                                     # len(val_set),
#                                     len(test_set)
#                                     )
#       )

train set: 308
test set: 35


In [ ]:
# gen_dataset_path='GenDataset/'
# !rm -r {gen_dataset_path}
# !mkdir {gen_dataset_path}
# !mkdir {gen_dataset_path+'Images/'}
# !mkdir {gen_dataset_path+'Masks_masses/'}
# !mkdir {gen_dataset_path+'Masks_cal/'}

In [ ]:
# PreprocessDicom(train_set, 1024, 8, DatasetPath, gen_dataset_path)

  0%|          | 0/308 [00:00<?, ?it/s]

In [ ]:
# fig = plt.figure(figsize=(20, 20))
# id=random.choice(glob('/content/GenDataset/Images/*')).split("/")[-1]


# fig.add_subplot(1,3,1)
# plt.imshow(dicom.dcmread(gen_dataset_path+'Images/'+id).pixel_array, cmap='gray')
# plt.gca().set_title('Image')

# fig.add_subplot(1,3,2)
# masses_mask=dicom.dcmread(gen_dataset_path+'Masks_masses/'+id).pixel_array
# plt.imshow(masses_mask, cmap='binary_r')
# plt.gca().set_title(f'Nb Masses:{len(np.unique(masses_mask))-1}')

# # if(len(val_masks[id]!=0)):
# #   cnt=0
# #   for box in val_boxes[id]:
# #     x=box[0]
# #     y=box[1]
# #     w=box[2]
# #     h=box[3]  
# #     im=cv2.rectangle(val_imges[id], (x,y),(w+x,h+y),(0,255,0),1)
# #     cnt+=1


# fig.add_subplot(1,3,3)
# calcifications_mask=dicom.dcmread(gen_dataset_path+'Masks_cal/'+id).pixel_array
# plt.imshow(calcifications_mask, cmap='binary_r')
# plt.gca().set_title(f'Nb Calcifications:{len(np.unique(calcifications_mask))-1}')


# plt.show()

In [ ]:
# -------------------------------------